In [12]:
import requests
import argparse
import time
import json
import StringIO
import gzip
import csv
import codecs
import pandas as pd

from bs4 import BeautifulSoup

In [13]:
df_amazon = pd.DataFrame(["url","content"])
df_apple = pd.DataFrame(["url","content"])
df_google = pd.DataFrame(["url","content"])
df_facebook = pd.DataFrame(["url","content"])
df_uber = pd.DataFrame(["url","content"])

target_size = 5
count_amz= 0
count_app= 0
count_ggl= 0
count_fb= 0
count_ub= 0

In [16]:

domain = "techcrunch.com"

# list of available indices
# index_list = ["2014-52","2015-06","2015-11","2015-14","2015-18","2015-22","2015-27"]

index_list = ["2019-13"]

#
# Searches the Common Crawl Index for a domain.
#
def search_domain(domain):

    record_list = []
    
    print "[*] Trying target domain: %s" % domain
    
    for index in index_list:
        
        print "[*] Trying index %s" % index
        
        cc_url  = "http://index.commoncrawl.org/CC-MAIN-%s-index?" % index
        cc_url += "url=%s&matchType=domain&output=json" % domain
        
        response = requests.get(cc_url)
        
        if response.status_code == 200:
            
            records = response.content.splitlines()
            
            for record in records:
                record_list.append(json.loads(record))
            
            print "[*] Added %d results." % len(records)
            
    
    print "[*] Found a total of %d hits." % len(record_list)
    
    return record_list        

#
# Downloads a page from Common Crawl - adapted graciously from @Smerity - thanks man!
# https://gist.github.com/Smerity/56bc6f21a8adec920ebf
#
def download_page(record):

    offset, length = int(record['offset']), int(record['length'])
    offset_end = offset + length - 1

    # We'll get the file via HTTPS so we don't need to worry about S3 credentials
    # Getting the file on S3 is equivalent however - you can request a Range
    prefix = 'https://commoncrawl.s3.amazonaws.com/'
    
    # We can then use the Range header to ask for just this set of bytes
    resp = requests.get(prefix + record['filename'], headers={'Range': 'bytes={}-{}'.format(offset, offset_end)})
    
    # The page is stored compressed (gzip) to save space
    # We can extract it using the GZIP library
    raw_data = StringIO.StringIO(resp.content)
    f = gzip.GzipFile(fileobj=raw_data)
    
    # What we have now is just the WARC response, formatted:
    data = f.read()
    
    response = ""
    
    if len(data):
        try:
            warc, header, response = data.strip().split('\r\n\r\n', 2)
        except:
            pass
            
    return response

#
# Extract links from the HTML  
#
def extract_external_links(html_content,url):
    
    try:

        parser = BeautifulSoup(html_content)

        # kill all script and style elements
        for script in parser(["script", "style"]):
            script.decompose()

        page_body = parser.find("body").get_text()

        if "amazon" in page_body.lower():
            df_amazon.append([url, page_body])
            count_amz += 1
            print "amazon::"+ count_amz + ":" +url
            
        if "apple" in page_body.lower():
            df_apple.append([url, page_body])
            count_app += 1
            print "apple::"+ count_app + ":" +url
            
        if "google" in page_body.lower() or "google" in url:
            df_google.append([url, page_body])
            count_ggl += 1
            print "google::"+ count_ggl + ":" +url
            
        if "facebook" in page_body.lower():
            df_facebook.append([url, page_body])
            count_fb += 1
            print "facebook::"+ count_fb + ":" +url
            
        if "uber" in page_body.lower():
            df_uber.append([url, page_body])
            count_ub += 1
            print "uber::"+ count_ub + ":" +url
            
    except:
        pass
    
    



record_list = search_domain(domain)
link_list   = []

for record in record_list:
    
    html_content = download_page(record)
    
    print "[*] Retrieved %d bytes for %s" % (len(html_content),record['url'])
    
    if len(html_content)>0 and (
        count_amz <target_size or
        count_app <target_size  or
        count_ggl <target_size  or
        count_fb <target_size  or
        count_ub <target_size):
    
        extract_external_links(html_content,record['url'])



[*] Trying target domain: techcrunch.com
[*] Trying index 2019-13
[*] Added 12457 results.
[*] Found a total of 12457 hits.
[*] Retrieved 905375 bytes for https://techcrunch.com/
[*] Retrieved 941431 bytes for https://techcrunch.com/
[*] Retrieved 176 bytes for http://www.techcrunch.com/
[*] Retrieved 0 bytes for https://www.techcrunch.com/
[*] Retrieved 964433 bytes for https://techcrunch.com/
[*] Retrieved 176 bytes for http://www.techcrunch.com/
[*] Retrieved 0 bytes for https://www.techcrunch.com/
[*] Retrieved 979187 bytes for https://techcrunch.com/
[*] Retrieved 176 bytes for http://www.techcrunch.com/
[*] Retrieved 0 bytes for https://www.techcrunch.com/
[*] Retrieved 176 bytes for http://www.techcrunch.com/
[*] Retrieved 970847 bytes for https://techcrunch.com/
[*] Retrieved 0 bytes for https://www.techcrunch.com/
[*] Retrieved 176 bytes for http://www.techcrunch.com/
[*] Retrieved 0 bytes for https://www.techcrunch.com/
[*] Retrieved 980612 bytes for https://techcrunch.com/
[

[*] Retrieved 65969 bytes for https://techcrunch.com/2005/08/23/google-talk-live-functional/
[*] Retrieved 62191 bytes for https://techcrunch.com/2005/08/26/flock-social-browsing-is-cool/
[*] Retrieved 62199 bytes for https://techcrunch.com/2005/08/26/flock-social-browsing-is-cool/
[*] Retrieved 68269 bytes for https://techcrunch.com/2005/08/29/attensa-the-smart-rss-reader/
[*] Retrieved 176 bytes for http://techcrunch.com/2005/09/01/technorati-releases-major-new-feature/
[*] Retrieved 57864 bytes for https://techcrunch.com/2005/09/01/technorati-releases-major-new-feature/
[*] Retrieved 57864 bytes for https://techcrunch.com/2005/09/01/technorati-releases-major-new-feature/
[*] Retrieved 54621 bytes for https://techcrunch.com/2005/09/04/pocketmod-type-stuff-and-print-it/
[*] Retrieved 176 bytes for http://www.techcrunch.com/2005/09/07/85-of-college-students-use-facebook/
[*] Retrieved 0 bytes for https://www.techcrunch.com/2005/09/07/85-of-college-students-use-facebook/
[*] Retrieved 6

KeyboardInterrupt: 

In [ ]:
https://techcrunch.com/2005/11/15/google-base-launched-yuck/

In [ ]:
    

# print "[*] Total external links discovered: %d" % len(link_list)

# with codecs.open("%s-links.csv" % domain,"wb",encoding="utf-8") as output:

#     fields = ["URL"]
    
#     logger = csv.DictWriter(output,fieldnames=fields)
#     logger.writeheader()
    
#     for link in link_list:
#         logger.writerow({"URL":link})

In [20]:
"google" in "https://techcrunch.com/2005/08/23/google-talk-live-functional/"

True